In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
input_width = 28
input_height = 28
input_channels = 1
n_input = 784
n_conv1 = 32
n_conv2 = 64
conv1_k = 5
conv2_k = 5
n_hidden = 1024
n_out = 10
pooling_window_size = 2

In [5]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k ,input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k ,n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_width//4 * input_height//4 * n_conv2, n_hidden])),
    "wout": tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bout": tf.Variable(tf.random_normal([n_out]))
}

In [6]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides=[1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    return tf.nn.relu(out)

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding="SAME", ksize=[1, k, k, 1], strides=[1, k, k, 1])

In [7]:
def cnn(x, weights, biases):
    x = tf.reshape(x, shape = [-1,input_height, input_width, 1])
    conv1 = conv(x, weights["wc1"], biases["bc1"])
    conv1 = maxpooling(conv1, k = pooling_window_size)
    
    conv2 = conv(conv1, weights["wc2"], biases["bc2"])
    conv2 = maxpooling(conv2, k = pooling_window_size)
    
    hidden_input = tf.reshape(conv2, shape=[-1, input_width//4 * input_height//4 * n_conv2])
    hidden_output = tf.add(tf.matmul(hidden_input, weights["wh1"]), biases['bh1'])
    hidden_output = tf.nn.relu(hidden_output)
    
    out =  tf.add(tf.matmul(hidden_output, weights["wout"]), biases['bout'])
    return out

In [8]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_out])
pred = cnn(x, weights, biases)

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimize = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _ , c = sess.run([optimize, cost], feed_dict={x:batch_x, y:batch_y})
        total_cost += c
    print(total_cost)

In [19]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)